In [1]:
# 导入必要的库
import json
import time
import torch
import os
from transformers import AutoTokenizer, AutoModel, AutoConfig
from langchain.llms.base import LLM
from typing import List, Optional, Dict

# 自定义 GLM 模型类，兼容 LangChain 的 LLM 接口
class GLM(LLM):
    max_length: int = 131072  # 设置模型的最大输入长度
    max_new_tokens: int = 20000  # 设置生成的最大新标记数
    temperature: float = 0.3
    top_p: float = 0.9
    tokenizer: object = None
    model: object = None
    llm_device: str = "cuda"  # 显式定义 llm_device

    def __init__(self, llm_device="cuda", **kwargs):
        super().__init__(**kwargs)
        self.llm_device = llm_device  # 直接设置 llm_device

    @property
    def _llm_type(self) -> str:
        return "GLM"

    def load_model(self, model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(
            model_name_or_path, trust_remote_code=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path, trust_remote_code=True
        )
        self.model = (
            AutoModel.from_pretrained(
                model_name_or_path,
                config=model_config,
                trust_remote_code=True,
            )
            .half()
            .to(self.llm_device)  # 使用 self.llm_device
        )
        self.model.eval()  # 确保模型处于推理模式

    def _call(
        self, prompt: str, stop: Optional[List[str]] = None, **kwargs
    ) -> str:
        try:
            response, _ = self.model.chat(
                self.tokenizer,
                prompt,
                history=[],  # 传递空历史以避免潜在问题
                max_length=self.max_length,
                max_new_tokens=self.max_new_tokens,
                temperature=self.temperature,
                top_p=self.top_p,
            )
            return response.strip()
        except Exception as e:
            print(f"生成响应时出错：{e}")
            return ""

    # 使用 invoke 而不是 __call__
    def invoke(self, *args, **kwargs):
        return self._call(*args, **kwargs)

# 定义 Agent 类，包含 profile 属性
class Agent:
    def __init__(self, name, profile, model_name_or_path, llm_device):
        self.name = name
        self.profile = profile
        self.device = llm_device
        self.llm = GLM(llm_device=self.device)  # 传递 llm_device
        self.llm.load_model(
            model_name_or_path=model_name_or_path
        )
        self.own_history = []  # 每个智能体维护自己的发言历史

    def ask(self, input_goal, shared_history, user_input=None):
        # 构建 prompt
        prompt = f"{self.profile}\n"
        prompt += f"任务要求：{input_goal}\n"

        # 加入共享的历史记录
        if shared_history:
            history_text = "\n".join(
                [f"{item['agent']}: {item['message']}" for item in shared_history]
            )
            prompt += f"已有的讨论历史：\n{history_text}\n"
        else:
            prompt += "（目前还没有讨论历史）\n"

        # 特殊处理蓝色思考帽(管理)
        if self.name == '蓝色思考帽(管理)':
            prompt += (
                "请总结各个思考帽的讨论，得出最终的答案。\n"
                "请注意，假定这个问题的最后结果是一个数字xx，请你的输出格式添加一个[#### ]，按照'[#### 数字xx]'进行输出。\n"
            )
        else:
            prompt += (
                "请根据你的角色，针对任务要求进行分析和讨论，最后输出一个明确的结论。\n"
            )

        prompt += f"{self.name}请继续发言："

        # 生成响应
        response = self.llm.invoke(prompt)
        return response.strip()

# 创建智能体
def create_agents(model_path, default_profiles):
    agents = {}
    cuda_devices = [f'cuda:{i}' for i in range(torch.cuda.device_count())]
    role_names = ["白色思考帽(客观)", "绿色思考帽(创新)", "黑色思考帽(批判)", "黄色思考帽(积极)", "红色思考帽(直觉)", "蓝色思考帽(管理)"]
    for i, name in enumerate(role_names):
        profile = default_profiles.get(name, f"你是{name}。请定义你的角色。")
        agent = Agent(
            name=name,
            profile=profile,
            model_name_or_path=model_path,
            llm_device=cuda_devices[i % len(cuda_devices)]
        )
        agents[name] = agent
    return agents

# 定义智能体的角色描述
default_profiles = {
    "白色思考帽(客观)": "你是白色思考帽(客观)。你的任务是客观地陈述事实和数据，说明题目条件的范围边界。请在分析中提供相关的事实和数据，仔细的解决这个数学题，必须得出答案（注意不能添加或改变题目条件，只能严格依据题目条件，参考其他帽子的解法，给出你自己重新分析出的不同的解题过程和方法）。",
    "绿色思考帽(创新)": "你是绿色思考帽(创新)。你的任务是创造性思考，提出新颖的观点和解决方案。根据现有的回答和易错点陷阱分析，仔细的解决这个数学题，必须得出答案（注意不能添加或改变题目条件，只能严格依据题目条件，参考其他帽子的解法，给出你自己重新分析出的不同的解题过程和方法）。",
    "黑色思考帽(批判)": "你是黑色思考帽(批判)。你的任务是批判性地分析问题，找到之前计算方式的不足和问题。找到问题的易错点和容易搞错的陷阱。仔细的解决这个数学题，必须得出答案（注意不能添加或改变题目条件，只能严格依据题目条件，参考其他帽子的解法，给出你自己重新分析出的不同的解题过程和方法）。",
    "黄色思考帽(积极)": "你是黄色思考帽(积极)。你的任务是积极地考虑问题，利用现有的信息进行解题，展示计算过程，仔细的解决这个数学题，必须得出答案（注意不能添加或改变题目条件，只能严格依据题目条件，参考其他帽子的解法，给出你自己重新分析出的不同的解题过程和方法）。",
    "红色思考帽(直觉)": "你是红色思考帽(直觉)。你的任务是根据直觉表达你对于这个问题关键信息的判断。找到最关键的题目条件和核心关键，仔细的解决这个数学题，必须得出答案（注意不能添加或改变题目条件，只能严格依据题目条件，参考其他帽子的解法，给出你自己重新分析出的不同的解题过程和方法）。",
    "蓝色思考帽(管理)": "你是蓝色思考帽(管理)。你的任务是总结各个思考帽的讨论，得出最终的答案。假定这个问题的最后结果是一个数字xx，请你的输出格式添加一个[#### ]，按照'[#### 数字xx]'进行输出。"
}

# 指定模型路径
model_path = "/data1/dxw_data/llm/chatglm3-6b-128k"

# 初始化智能体
agents = create_agents(model_path, default_profiles)


2024-10-18 20:39:46.665882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-18 20:39:46.790707: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 20:39:47.358099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-10-18 20:39:47.358160: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:


# 读取 test.jsonl 文件并提取问题列表
def read_test_file(file_path):
    questions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            question = data['question']
            questions.append(question)
    return questions

# 运行智能体之间的对话
def run_conversation(agents, question, conversation_history):
    role_histories = {name: [] for name in agents.keys()}
    
    # 指定的思考帽顺序
    hat_order = ['白色思考帽(客观)', '红色思考帽(直觉)', '黄色思考帽(积极)', '黑色思考帽(批判)', '绿色思考帽(创新)']
    
    # 依次让每个思考帽发言
    for hat in hat_order:
        if hat in agents:
            current_agent = agents[hat]
            # print(f"\n轮到 {hat} 发言:")
            response = current_agent.ask(question, conversation_history)
            # print(f"{hat}: {response}")
            conversation_history.append({
                'agent': current_agent.name,
                'message': response
            })
            role_histories[current_agent.name].append(response)
        else:
            print(f"{hat} 不在智能体列表中，跳过。")
    
    # 蓝色思考帽(管理)总结并输出最终答案
    blue_agent = agents['蓝色思考帽(管理)']
    # print(f"\n轮到 蓝色思考帽(管理) 发言:")
    blue_response = blue_agent.ask(question, conversation_history)
    # print(f"蓝色思考帽(管理): {blue_response}")
    conversation_history.append({
        'agent': blue_agent.name,
        'message': blue_response
    })
    role_histories[blue_agent.name].append(blue_response)
    
    return role_histories

# 设计 eval_blue_results 函数，提取蓝色思考帽的最终结果
def eval_blue_results(question, blue_response):
    import re
    # 使用正则表达式提取 #### 后的数字
    match = re.search(r'#### \s*([\d\.\-]+)', blue_response)
    if match:
        final_result = match.group(1)
    else:
        final_result = None
    return {
        'question': question,
        'blue_response': blue_response,
        'final_result': final_result
    }

# 处理问题并保存结果
def process_questions(agents, questions):
    results = []
    batch_size = 10
    output_file = '/data1/dxw_data/llm/six_hats/metric_analysis/blue_agent_results.jsonl'
    
    for i, question in enumerate(questions):
        # print(f"\n------------------------------正在处理第 {i+1} 个问题：{question}-----------------------------------")
        conversation_history = []
        role_histories = run_conversation(agents, question, conversation_history)
        
        # 获取蓝色思考帽的最终回答
        blue_response = role_histories['蓝色思考帽(管理)'][-1]
        
        # 评估蓝色思考帽的回答
        result = eval_blue_results(question, blue_response)
        print(f"\n------------------------------第 {i+1} 个问题的回答是：{result}-------------------------")
        
        results.append(result)
        
        # 每处理10个问题，保存一次结果
        if (i + 1) % batch_size == 0:
            # 追加写入到文件
            with open(output_file, 'a', encoding='utf-8') as f:
                for res in results:
                    json_line = json.dumps(res, ensure_ascii=False)
                    f.write(json_line + '\n')
            # 清空结果列表
            results = []
    
    # 保存剩余的结果
    if results:
        with open(output_file, 'a', encoding='utf-8') as f:
            for res in results:
                json_line = json.dumps(res, ensure_ascii=False)
                f.write(json_line + '\n')
    print(f"\n所有问题已处理完毕，结果已保存到 {output_file}")

# 主程序
if __name__ == "__main__":
    test_file_path = "/data1/dxw_data/llm/six_hats/metric_analysis/dataset/grade-school-math/grade_school_math/data/test.jsonl"
    questions = read_test_file(test_file_path)
    process_questions(agents, questions)

Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



------------------------------正在处理第 1 个问题：Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?-----------------------------------

轮到 白色思考帽(客观) 发言:


Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


白色思考帽(客观): 根据题目条件，我们可以得知以下信息：

- Janets ducks lay 16 eggs per day。
- 她每天早上吃三个鸡蛋。
- 她每天为朋友烘焙四个鸡蛋的面包。
- 她每天在农贸市场上以 $2 美元的价格出售剩余的鸡蛋。

现在我们要求的是她每天在农贸市场上赚多少钱。

我们可以用以下公式来计算她每天在农贸市场上赚的钱：

每天在农贸市场上赚的钱 = 出售剩余鸡蛋的数量 x 每个鸡蛋的价格

首先，我们来计算出售剩余鸡蛋的数量。题目告诉我们，她每天吃三个鸡蛋，烘焙四个鸡蛋的面包，因此她每天出售剩余的鸡蛋数量为：

出售剩余鸡蛋的数量 = 16 - 3 - 4 = 9

接下来，我们来计算每个鸡蛋的价格。题目告诉我们，她每天以 $2 美元的价格出售剩余的鸡蛋，因此每个鸡蛋的价格为：

每个鸡蛋的价格 = 2 美元

现在我们已经知道出售剩余鸡蛋的数量和每个鸡蛋的价格，我们可以用以下公式来计算她每天在农贸市场上赚的钱：

每天在农贸市场上赚的钱 = 出售剩余鸡蛋的数量 x 每个鸡蛋的价格

每天在农贸市场上赚的钱 = 9 x 2 美元

每天在农贸市场上赚的钱 = 18 美元

因此，我们得出的结论是：Janet每天在农贸市场上赚 18 美元。

轮到 红色思考帽(直觉) 发言:


Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


蓝色思考帽(管理): 根据各个思考帽的分析，我们可以得出最终的答案是[#### 1]。这是因为白色思考帽、红色思考帽、黄色思考帽和绿色思考帽的分析都认为Carlos在种植柠檬树后的第1年开始盈利，这是基于现实的成本和收入的计算得出的。同时，他们也都考虑到了柠檬树的生命周期和潜在问题。因此，最终的答案是[#### 1]。

------------------------------第 13 个问题的回答是：{'question': 'Carlos is planting a lemon tree. The tree will cost $90 to plant. Each year it will grow 7 lemons, which he can sell for $1.5 each. It costs $3 a year to water and feed the tree. How many years will it take before he starts earning money on the lemon tree?', 'blue_response': '根据各个思考帽的分析，我们可以得出最终的答案是[#### 1]。这是因为白色思考帽、红色思考帽、黄色思考帽和绿色思考帽的分析都认为Carlos在种植柠檬树后的第1年开始盈利，这是基于现实的成本和收入的计算得出的。同时，他们也都考虑到了柠檬树的生命周期和潜在问题。因此，最终的答案是[#### 1]。', 'final_result': '1'}-------------------------

------------------------------正在处理第 14 个问题：Melanie is a door-to-door saleswoman. She sold a third of her vacuum cleaners at the green house, 2 more to the red house, and half of what was left at the orange house. If Melanie has 5 vacuum cleaners left, how many did she start with?-------------------------

Both `max_new_tokens` (=20000) and `max_length`(=131072) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


白色思考帽(客观): 首先，根据题目描述，Melanie卖出了她真空清洁器的1/3，即3/5，在绿色房子。她还卖出了另外2台，即3/5 + 2/5 = 5/5 = 1台，在红色房子。最后，她卖出了剩下的一半，即3/5 * 1/2 = 3/10，在橙色房子。

现在，我们需要找到Melanie最初卖出了多少台真空清洁器。我们可以通过以下步骤来解决这个数学题：

1. 计算Melanie卖出的总台数：3/5 + 1/5 + 3/10 = 3.2台

2. 计算Melanie最初卖出的总台数：3.2台 = 32/10 = 32/5

3. 由于Melanie最初卖出的总台数必须是一个整数，因此我们需要将32/5向下取整，得到她最初卖出了6台真空清洁器。

因此，结论是：Melanie最初卖出了6台真空清洁器。

轮到 红色思考帽(直觉) 发言:


KeyboardInterrupt: 